# Vertex AI - Training Job and Serving

This notebook creates a python package for a TensorFlow training project that uses the `<PROJECT_ID>.digits.digits_prepped` BigQuery table. Vertex AI clients are used to setup a Vertex AI custom training job to run the training package.  Then Vertex AI clients are used to upload the model and deploy it to an endpoint for online predictions.

**Prerequisites**
- `00 - Initial Setup`
- `01 - BigQuery - Data`
- `04 - Vertex AI - Notebook` (helpful to understand)
    - the model created here is stored in a python package in this notebook
    
**Resources**
- Adopted From:
    - https://github.com/GoogleCloudPlatform/ai-platform-samples/blob/master/ai-platform-unified/notebooks/custom_job_image_classification_model_for_online_prediction.ipynb
- Using Google Cloud Client Libraries (Python for Vertex AI):
    - https://googleapis.dev/python/aiplatform/latest/index.html

**Overview**

<img src="architectures/statmike-mlops-05.png">

---
## Setup

Setup the environment:

In [1]:
from google.cloud import aiplatform
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value
from datetime import datetime

Define Parameters:

In [2]:
# Locations
REGION = 'us-central1'
PROJECT_ID='statmike-mlops'
BUCKET_NAME='gs://{}/digits/model/05_aip_train_job'.format(PROJECT_ID)
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
JOB_NAME='05_AIP_DIGITS_'+TIMESTAMP
MODEL_DIR = '{}/{}'.format(BUCKET_NAME, JOB_NAME)
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

# Resources
TRAIN_IMAGE='us-docker.pkg.dev/cloud-aiplatform/training/tf-cpu.2-4:latest'
DEPLOY_IMAGE ='us-docker.pkg.dev/cloud-aiplatform/prediction/tf2-cpu.2-3:latest'
TRAIN_COMPUTE='n1-standard-4'
DEPLOY_COMPUTE='n1-standard-4'

# TF Parameters to pass
EPOCHS = 25
BATCH_SIZE = 30

Setup AI Platform Python Clients
- https://googleapis.dev/python/aiplatform/latest/index.html

In [3]:
API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)
client_options = {"api_endpoint": API_ENDPOINT}
clients = {}

---
## Training Job

Create a client for the AIP Job Service:

In [4]:
clients['job'] = aiplatform.gapic.JobServiceClient(client_options=client_options)

Helper functions wrappers for interacting with the client: create, list, get, cancel

In [5]:
def create_custom_job(custom_job):
    response = clients['job'].create_custom_job(parent=PARENT, custom_job=CUSTOM_JOB)
    print("name:", response.name)
    print("display_name:", response.display_name)
    print("state:", response.state)
    print("create_time:", response.create_time)
    print("update_time:", response.update_time)
    return response.name

def list_custom_jobs():
    response = clients['job'].list_custom_jobs(parent=PARENT)
    for job in response:
        print(response)
        
def get_custom_job(name, silent=False):
    response = clients['job'].get_custom_job(name=name)
    if silent:
        return response
    print("name:", response.name)
    print("display_name:", response.display_name)
    print("state:", response.state)
    print("create_time:", response.create_time)
    print("update_time:", response.update_time)
    return response

def cancel_job(name):
    try:
        response = clients['job'].cancel_custom_job(name=name)
        print(response)
    except Exception as e:
        print(e)

Define job parameters to pass to the client:

In [6]:
MACHINE_SPEC = {
    "machine_type": TRAIN_COMPUTE,
    "accelerator_count": 0
}


CMDARGS = [
    "--epochs=" + str(EPOCHS),
    "--batch_size=" + str(BATCH_SIZE)
]

WORKER_POOL_SPEC = [
    {
        "replica_count": 1,
        "machine_spec": MACHINE_SPEC,
        "python_package_spec": {
            "executor_image_uri": TRAIN_IMAGE,
            "package_uris": [BUCKET_NAME + "/trainer_cifar.tar.gz"],
            "python_module": "trainer.task",
            "args": CMDARGS
        }
    }
]

JOB_SPEC = {
    "worker_pool_specs": WORKER_POOL_SPEC,
    "base_output_directory": {"output_uri_prefix": MODEL_DIR}
    
}

CUSTOM_JOB = {
    "display_name": JOB_NAME,
    "job_spec": JOB_SPEC
}

Assemble the Python training package:

In [7]:
!rm -rf custom
!mkdir custom


setup_py = "from setuptools import setup\n\
if __name__ == '__main__':\n\
    setup()"

setup_py = "import setuptools\n\
REQUIRED_PACKAGES = ['tensorflow_io']\n\
setuptools.setup(\n\
    name='trainer',\n\
    version='0.1',\n\
    #install_requires=REQUIRED_PACKAGES,\n\
    packages=setuptools.find_packages(),\n\
    #include_package_data=True,\n\
    description='Digit Training Package')"

!echo "$setup_py" > custom/setup.py

!mkdir custom/trainer
!touch custom/trainer/__init__.py

E0706 14:28:39.726228003      72 backup_poller.cc:133]       Run client channel backup poller: {"created":"@1625581719.726114584","description":"pollset_work","file":"src/core/lib/iomgr/ev_epollex_linux.cc","file_line":321,"referenced_errors":[{"created":"@1625581719.726105773","description":"Bad file descriptor","errno":9,"file":"src/core/lib/iomgr/ev_epollex_linux.cc","file_line":952,"os_error":"Bad file descriptor","syscall":"epoll_wait"}]}


In [8]:
%%writefile custom/trainer/task.py
from tensorflow_io.bigquery import BigQueryClient
from tensorflow_io.bigquery import BigQueryReadSession
import tensorflow as tf
from google.cloud import bigquery
import argparse
import os
import sys

parser = argparse.ArgumentParser()
# the passed param, dest: a name for the param, default: if absent fetch this param from the OS, type: type to convert to, help: description of argument
parser.add_argument('--model-dir', dest='model_dir', default=os.getenv("AIP_MODEL_DIR"), type=str, help='Model dir.')
parser.add_argument('--epochs',dest='epochs', default=10, type=int, help='Number of Epochs')
parser.add_argument('--batch_size',dest='batch_size', default=32, type=int, help='Batch Size')
#parser.add_argument('',dest='', default=, type=, help='')
args = parser.parse_args()

# built in parameters for data source:
PROJECT_ID='statmike-mlops'
BQDATASET_ID='digits'
BQTABLE_ID='digits_prepped'

selected_fields = ['p0', 'p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'p7', 'p8', 'p9', 'p10', 'p11', 'p12', 'p13', 'p14', 'p15', 'p16', 'p17', 'p18', 'p19', 'p20', 'p21', 'p22', 'p23', 'p24', 'p25', 'p26', 'p27', 'p28', 'p29', 'p30', 'p31', 'p32', 'p33', 'p34', 'p35', 'p36', 'p37', 'p38', 'p39', 'p40', 'p41', 'p42', 'p43', 'p44', 'p45', 'p46', 'p47', 'p48', 'p49', 'p50', 'p51', 'p52', 'p53', 'p54', 'p55', 'p56', 'p57', 'p58', 'p59', 'p60', 'p61', 'p62', 'p63', 'target']
output_types = ['FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'INT64']

feature_columns = []
feature_layer_inputs = {}
for header in selected_fields:
    if header != 'target':
        feature_columns.append(tf.feature_column.numeric_column(header))
        feature_layer_inputs[header] = tf.keras.Input(shape=(1,),name=header)

from tensorflow.python.framework import dtypes
output_types = [dtypes.float64 if x=='FLOAT64' else dtypes.int64 for x in output_types]

def transTable(row_dict):
    target=row_dict.pop('target')
    target = tf.one_hot(tf.cast(target,tf.int64),10)
    target = tf.cast(target,tf.float32)
    return(row_dict,target)

client = BigQueryClient()
session = client.read_session("projects/"+PROJECT_ID,PROJECT_ID,BQTABLE_ID,BQDATASET_ID,selected_fields,output_types,row_restriction="SPLITS='TRAIN'",requested_streams=3)
table = session.parallel_read_rows()
table = table.map(transTable)
train = table.shuffle(100000).batch(args.batch_size)

client = BigQueryClient()
session = client.read_session("projects/"+PROJECT_ID,PROJECT_ID,BQTABLE_ID,BQDATASET_ID,selected_fields,output_types,row_restriction="SPLITS='TEST'",requested_streams=3)
table = session.parallel_read_rows()
table = table.map(transTable)
test = table.batch(args.batch_size)

feature_layer = tf.keras.layers.DenseFeatures(feature_columns)
feature_layer_outputs = feature_layer(feature_layer_inputs)
model = tf.keras.Model(inputs=[v for v in feature_layer_inputs.values()],outputs=tf.keras.layers.Dense(10,activation=tf.nn.softmax)(feature_layer_outputs))
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
tf.keras.utils.plot_model(model,show_shapes=True, show_dtype=True)

history = model.fit(train,epochs=args.epochs)

model.save(args.model_dir)

Writing custom/trainer/task.py


Store the training package in a Cloud Storage Bucket:

In [9]:
!rm -f custom.tar custom.tar.gz
!tar cvf custom.tar custom
!gzip custom.tar
!gsutil cp custom.tar.gz $BUCKET_NAME/trainer_cifar.tar.gz

E0706 14:29:09.726237545      88 backup_poller.cc:133]       Run client channel backup poller: {"created":"@1625581749.726085150","description":"pollset_work","file":"src/core/lib/iomgr/ev_epollex_linux.cc","file_line":321,"referenced_errors":[{"created":"@1625581749.726074808","description":"Bad file descriptor","errno":9,"file":"src/core/lib/iomgr/ev_epollex_linux.cc","file_line":952,"os_error":"Bad file descriptor","syscall":"epoll_wait"}]}
custom/
custom/setup.py
custom/trainer/
custom/trainer/task.py
custom/trainer/__init__.py
Copying file://custom.tar.gz [Content-Type=application/x-tar]...
/ [1 files][  1.6 KiB/  1.6 KiB]                                                
Operation completed over 1 objects/1.6 KiB.                                      


Submit the training job:

In [10]:
JOB_ID = create_custom_job(CUSTOM_JOB)

name: projects/691911073727/locations/us-central1/customJobs/4009386742871228416
display_name: 05_AIP_DIGITS_20210706142708
state: JobState.JOB_STATE_PENDING
create_time: 2021-07-06 14:29:19.069452+00:00
update_time: 2021-07-06 14:29:19.069452+00:00


Get information as the job runs:

In [24]:
trainjob_response = get_custom_job(JOB_ID)

name: projects/691911073727/locations/us-central1/customJobs/4009386742871228416
display_name: 05_AIP_DIGITS_20210706142708
state: JobState.JOB_STATE_SUCCEEDED
create_time: 2021-07-06 14:29:19.069452+00:00
update_time: 2021-07-06 14:40:37.872504+00:00


---
## Serving

---
### Upload the Model

Check that model training was successful (and update the path to the model store):

In [25]:
if trainjob_response.state == aiplatform.gapic.JobState.JOB_STATE_SUCCEEDED:
    MODEL_DIR = MODEL_DIR + "/model"
    
print(MODEL_DIR)

gs://statmike-mlops/digits/model/05_aip_train_job/05_AIP_DIGITS_20210706142708/model


Create a client to the Model Service:

In [26]:
clients['model'] = aiplatform.gapic.ModelServiceClient(client_options=client_options)

Upload the model using the client:

In [27]:
MODEL = {
    "display_name": trainjob_response.display_name,
    "metadata_schema_uri": "",
    "artifact_uri": MODEL_DIR,
    "container_spec": {
        "image_uri": DEPLOY_IMAGE,
        "command": [],
        "args": [],
        "env": [{"name": "env_name", "value": "env_value"}],
        "ports": [{"container_port": 8080}],
        "predict_route": "",
        "health_route": ""
    }
}

uploaded_model = clients['model'].upload_model(parent=PARENT, model=MODEL)

Review the uploaded models information:

In [28]:
model_info = clients['model'].get_model(name=uploaded_model.result(timeout=180).model)
model_info.name

'projects/691911073727/locations/us-central1/models/4339649249529561088'

---
### Endpoint Creation

Create a client to the endpoint service:

In [29]:
clients['endpoint'] = aiplatform.gapic.EndpointServiceClient(client_options=client_options)

Create the endpoint:

In [30]:
ENDPOINT_NAME = 'ENDPOINT_'+JOB_NAME
endpoint = clients['endpoint'].create_endpoint(parent=PARENT, endpoint={"display_name": ENDPOINT_NAME})

In [31]:
endpoint_info = clients['endpoint'].get_endpoint(name=endpoint.result(timeout=180).name)
endpoint_info.name

'projects/691911073727/locations/us-central1/endpoints/5819970132516012032'

---
### Deploy Model to Endpoint

Setup Deployment Parameters:

In [32]:
MACHINE_SPEC = {
    "machine_type": DEPLOY_COMPUTE,
    "accelerator_count": 0,
}
DMODEL = {
        "model": model_info.name,
        "display_name": 'DEPLOYED_'+JOB_NAME,
        "dedicated_resources": {
            "min_replica_count": 1,
            "max_replica_count": 2,
            "machine_spec": MACHINE_SPEC
        }   
}
TRAFFIC = {
    '0' : 100
}

Deploy the Model to the Endpoint:

In [33]:
dmodel = clients['endpoint'].deploy_model(endpoint=endpoint_info.name, deployed_model=DMODEL, traffic_split=TRAFFIC)

In [34]:
dmodel_info = dmodel.result().deployed_model
dmodel_info.id

'4618986775635820544'

In [35]:
clients['endpoint'].get_endpoint(name=endpoint_info.name)

name: "projects/691911073727/locations/us-central1/endpoints/5819970132516012032"
display_name: "ENDPOINT_05_AIP_DIGITS_20210706142708"
deployed_models {
  id: "4618986775635820544"
  model: "projects/691911073727/locations/us-central1/models/4339649249529561088"
  display_name: "DEPLOYED_05_AIP_DIGITS_20210706142708"
  create_time {
    seconds: 1625582511
    nanos: 621968000
  }
  dedicated_resources {
    machine_spec {
      machine_type: "n1-standard-4"
    }
    min_replica_count: 1
    max_replica_count: 2
  }
}
traffic_split {
  key: "4618986775635820544"
  value: 100
}
etag: "AMEw9yMuZH3v75SbvDkHR6qYOQHqA-yUrDB7JIqh86a-hL52Hc1PhLSNzu3waoutyJb0"
create_time {
  seconds: 1625582503
  nanos: 740357000
}
update_time {
  seconds: 1625582817
  nanos: 907513000
}

---
## Prediction

Create a client to the prediction service:

In [36]:
clients['prediction'] = aiplatform.gapic.PredictionServiceClient(client_options=client_options)

Setup an observation for prediction:

In [37]:
%%bigquery pred
SELECT *
FROM `digits.digits_prepped`
WHERE splits='TEST'

Downloading: 100%|██████████| 366/366 [00:01<00:00, 326.91rows/s]


In [38]:
pred.head(1)

,p0,p1,p2,p3,p4,p5,p6,p7,p8,p9,...,p57,p58,p59,p60,p61,p62,p63,target,target_OE,SPLITS
0,0.0,0.0,0.0,9.0,8.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,7.0,14.0,9.0,0.0,0.0,0,Even,TEST


In [39]:
newob = pred.loc[:0,'p0':'p63'].to_dict(orient='records')[0]
#newob

Request prediction from prediction service:

In [40]:
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value

response = clients['prediction'].predict(endpoint=endpoint_info.name, instances=[json_format.ParseDict(newob, Value())], parameters=json_format.ParseDict({}, Value()))

In [41]:
response.predictions

[[0.872015238, 0.000258201791, 3.30423244e-09, 5.97235109e-12, 0.0258183852, 3.31672226e-07, 0.101762742, 1.96920397e-10, 0.000144183723, 8.55353562e-07]]

In [42]:
import numpy as np
np.argmax(response.predictions[0])

0

---

## Remove Resources
- undeploy model
- delete endpoint
- remove model
- delete training > custom job
- delete python training application:
    - these could be used by later notebooks in this project:
    - delete custom and custom.tar.gz

Undeploy Model:

In [34]:
dmodel = clients['endpoint'].get_endpoint(name=endpoint_info.name).deployed_models[0].id
clients['endpoint'].undeploy_model(endpoint=endpoint_info.name, deployed_model_id=dmodel)

Delete Endpoint:

In [35]:
clients['endpoint'].delete_endpoint(name=endpoint_info.name)

Remove Model:

In [36]:
clients['model'].delete_model(name=model_info.name)

Delete Training > Custom Job:

In [38]:
clients['job'].delete_custom_job(name=trainjob_response.name)

Delete Model Files: 
- This includes the training package and the saved model
- These could be useful for other notebooks

In [49]:
from google.cloud import storage
gcs = storage.Client()

path = gcs.bucket(PROJECT_ID)
blobs = path.list_blobs(prefix='digits/aip_train_job')
for blob in blobs:
    print(blob)
    blob.delete()

<Blob: statmike-mlops, digits/aip_train_job/AIP_DIGITS_20210413130018/, 1618319425187771>
<Blob: statmike-mlops, digits/aip_train_job/AIP_DIGITS_20210413130018/model/, 1618319425350552>
<Blob: statmike-mlops, digits/aip_train_job/AIP_DIGITS_20210413130018/model/assets/, 1618319427738045>
<Blob: statmike-mlops, digits/aip_train_job/AIP_DIGITS_20210413130018/model/saved_model.pb, 1618319428158820>
<Blob: statmike-mlops, digits/aip_train_job/AIP_DIGITS_20210413130018/model/variables/, 1618319425484405>
<Blob: statmike-mlops, digits/aip_train_job/AIP_DIGITS_20210413130018/model/variables/variables.data-00000-of-00001, 1618319426990754>
<Blob: statmike-mlops, digits/aip_train_job/AIP_DIGITS_20210413130018/model/variables/variables.index, 1618319427197772>
<Blob: statmike-mlops, digits/aip_train_job/trainer_cifar.tar.gz, 1618318849607448>
